In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Systems Ltd.**

In [ ]:
sys = pd.read_csv('/kaggle/input/stock-data-avn-netsol-ptcl-sys-year-2018/Year_2018/SYS_Stock_data.csv') 
#The "SYS" file name can be changed to "NETSOL", "AVN" and "PTC".

#preprocessing data
sys['Time'] = pd.to_datetime(sys.Time) # correct the format of date
sys = sys.set_index('Time') # Set Time column as row index

sys.head()

# **Behaviour of 'CLOSE'**

We'll firstly see the behvaiour of 'Close' over the 2018 year.

> So it was increasing till 8th month and then a continue decrease in the stocks happened.

In [ ]:
sys['Close'].plot(legend = True, figsize=(15,10))

# **Trend indication by moving average method:**

In [ ]:
days = 50 # Moving average window
col_name = "mv_avg for " + str(days) + " days" # New column to store moving average vlues

sys[col_name] = sys['Close'].rolling(days).mean() #Calculating moving average

print(sys)

In [ ]:
sys[['Close', 'mv_avg for 50 days']].plot(figsize=(15,10)) # Plotting the closing price with moving avarage for comprison

# **Behaviour of 'DAILY RETURN'**

In [ ]:
daily_return = sys['Close'].pct_change() # Calculate the daily returns

sys['daily_return'] = daily_return # Create new column and assign daily return values to it

plt.ylabel('Percentage Change') # Assign a name to the y-axis of plot

sys['daily_return'].plot(legend = True, figsize=(15,10)) # plot the daily return values

we can see the daily return is varying between +0.04 to -0.04 which is not a really big range hence, the stocks are considered stable.

In [ ]:
plt.ylabel('Probability Density Value') # Assign a name to the y-axis of plot

sns.distplot(sys['daily_return'].dropna(), bins = 100, color = 'red') # plots a distribution graph of KDE and histogram 

shows that the higheset probability for the next daily return would be closer to zero i.e. the stock will remain the stable.

# **Finding Correlation between all companies**

In [ ]:
sys = pd.read_csv('/kaggle/input/stock-data-avn-netsol-ptcl-sys-year-2018/Year_2018/SYS_Stock_data.csv')
ns = pd.read_csv('/kaggle/input/stock-data-avn-netsol-ptcl-sys-year-2018/Year_2018/NETSOL_Stock_data.csv')
ptc = pd.read_csv('/kaggle/input/stock-data-avn-netsol-ptcl-sys-year-2018/Year_2018/PTCL_Stock_data.csv')
avn = pd.read_csv('/kaggle/input/stock-data-avn-netsol-ptcl-sys-year-2018/Year_2018/AVN_Stock_data.csv') 

sys['Time'] = pd.to_datetime(sys.Time)
ns['Time'] = pd.to_datetime(ns.Time)
ptc['Time'] = pd.to_datetime(ptc.Time)
avn['Time'] = pd.to_datetime(avn.Time)

sys = sys.set_index('Time')
ns = ns.set_index('Time')
ptc = ptc.set_index('Time')
avn = avn.set_index('Time')

# Correlation with 'CLOSE'

In [ ]:
df = pd.DataFrame({'SYS': sys['Close'], 
                   'NETSOL': ns['Close'], 
                   'PTC': ptc['Close'], 
                   'AVN': avn['Close']})

print("The New DataFrame\n", df.head())

corr = (df.dropna()).corr() # Calculating correlation after dropping null values

print("The correlations\n", corr)

In [ ]:
sns.heatmap(corr, annot = True) # Plotting heatmap

NETSOL and SYS has strong positive correlation with Close.

# Correlation with 'DAILY RETURN'

In [ ]:
all_returns = df.pct_change()

corr = (all_returns.dropna()).corr()
sns.heatmap(corr, annot = True)

NETSOL and AVN has a strong postive correlation with daily returns.

# **Risk  in investing:**

In [ ]:
ret = all_returns.dropna() # drop the null values

avg_daily_return = ret.mean() # Take mean of the daily return of all companies
print("Averge daily return of companies\n", avg_daily_return)

daily_risk = ret.std() # Take standard deviation of the daily return of all companies
print("\nDaily Risk or standard deviation of companies\n", daily_risk)

In [ ]:
plt.xlabel("Daily Average Expected Return") # Name the x-axis
plt.ylabel("Daily Risk") # Name the y-axis

plt.grid() # Add grid lines on the plot

plt.scatter(avg_daily_return, daily_risk, s = 30) # Plot the scatter plot for risk & return

In [ ]:
ret = all_returns.dropna()

avg_daily_return = ret.mean()
daily_risk = ret.std()

plt.xlabel("Daily Average Expected Return")
plt.ylabel("Daily Risk")

plt.xlim(ret.mean().min() + ret.mean().min()*2, ret.mean().max() + ret.mean().max()*2)

for label, x, y in zip(ret.columns, ret.mean(), ret.std()):
    plt.annotate(
        label,
        xy = (x,y), xytext = (50, 50),
        textcoords = 'offset points', ha = 'right', va = 'bottom',
        arrowprops = dict(arrowstyle = '<-', connectionstyle = 'arc3,rad=-0.4'))

plt.grid()

plt.scatter(avg_daily_return, daily_risk, s = 30)


So, from the above graph, the following results can be inferred:

* NETSOL stock has a little positive daily return and a very high risk, which means the loss is also high.
* Avanceon has a high positive daily return, and the risk value is also less than NETSOL.
* Systems Ltd has a positive daily return, but it is less than Avanceon. The risk value is less than NETSOL and Avanceon so it can be considered a good stock.
* The PTC stock has the lowest risk value, but its average daily return value is negative. This indicates that even though our losses would be little, there would not be a positive return for our investment.


# **Quantile method to calculate loss on any investment**

In [ ]:
df = pd.DataFrame({'SYS': sys['Close'], 
                   'NETSOL': ns['Close'], 
                   'PTC': ptc['Close'], 
                   'AVN': avn['Close']})

all_returns = df.pct_change()

investment = 100000

loss = (abs(all_returns.quantile(0.1))) * investment
print(loss)

Lowest loss for PTC for investment of 100000